In [6]:
dir_executable = '/usr/local/bin/geckodriver'
dir_html = 'file:///Users/yitingli/Desktop/EDGAR/FinDocNLP/parsers/Amber/2022.html'
import base64
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.remote.webelement import WebElement 

In [2]:



import itertools

try:
    driver = webdriver.Firefox(executable_path= dir_executable, service_log_path=None)
except Exception:
    driver = webdriver.Firefox(dir_webdriver)



driver.get(dir_html)

found_text = driver.find_elements(By.TAG_NAME, 'span')
# found_annotation = driver.find_elements(By.TAG_NAME, 'ix:nonnumeric')
temp_txt = [i for i in found_text if i.text not in set([' ', ''])]
annotation_dict = dict()
# have_annotation = np.zeros_like(temp_txt, 'bool')
# elements_with_annotation = []
for i in range(len(temp_txt)):
    found_annotation = temp_txt[i].find_elements(By.TAG_NAME, 'ix:nonnumeric')
    found_annotation += temp_txt[i].find_elements(By.TAG_NAME, 'ix:nonfraction')
    # if len(found_annotation) != 0:
        # have_annotation[i] = True
        # elements_with_annotation += [temp_txt[i]]
    annotation_dict[temp_txt[i]] = found_annotation
    





# temp_anno = [i for i in found_annotation if i.text not in set([' ', ''])]

def border_red(elem, driver):
    driver.execute_script(f"arguments[0].setAttribute(arguments[1], arguments[2])", elem, 
                          "style", "padding: 1px; border: 2px solid red; display: inline-block")

def border_blue(elem, driver):
    driver.execute_script(f"arguments[0].setAttribute(arguments[1], arguments[2])", elem, 
                          "style", "padding: 1px; border: 2px solid blue; display: inline-block")
def border_green(elem, driver):
    driver.execute_script(f"arguments[0].setAttribute(arguments[1], arguments[2])", elem, 
                          "style", "padding: 1px; border: 2px solid green; display: inline-block")
def border_yellow(elem, driver):
    driver.execute_script(f"arguments[0].setAttribute(arguments[1], arguments[2])", elem, 
                          "style", "padding: 1px; border: 2px solid yellow; display: inline-block")
def border_pink(elem, driver):
    driver.execute_script(f"arguments[0].setAttribute(arguments[1], arguments[2])", elem, 
                          "style", "padding: 1px; border: 2px solid pink; display: inline-block")


for i in temp_txt:
    border_red(i, driver);
for i in annotation_dict:
    for j in annotation_dict[i]:
        border_blue(j, driver)


# tables = [i.get_attribute('innerHTML') for i in driver.find_elements(By.TAG_NAME, 'table')]
found_table = driver.find_elements(By.TAG_NAME, 'table')


table_is_numeric = np.zeros_like(found_table, 'int')# 0: numerical, 1: non-numerical, 2: unannotated
for i in range(len(found_table)):
    try: 
        found_numeric = found_table[i].find_element(By.TAG_NAME, 'ix:nonfraction')
        table_is_numeric[i] = 0
    except NoSuchElementException:
        try:
            found_numeric = found_table[i].find_element(By.TAG_NAME, 'ix:nonnumeric')
            table_is_numeric[i] = 1
        except NoSuchElementException:
            table_is_numeric[i] = 2

for i in range(len(found_table)):
    if table_is_numeric[i] == 0:
        
        border_green(found_table[i], driver)
    elif table_is_numeric[i] == 1:
        border_yellow(found_table[i], driver)
    else:
        border_pink(found_table[i], driver)

/var/folders/w1/2dngyhy57h996jndj_5w9mn40000gn/T/ipykernel_11983/257382381.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path= dir_executable, service_log_path=None)
/var/folders/w1/2dngyhy57h996jndj_5w9mn40000gn/T/ipykernel_11983/257382381.py:4: DeprecationWarning: service_log_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path= dir_executable, service_log_path=None)


1666727827446	geckodriver	INFO	Listening on 127.0.0.1:54444
1666727827960	mozrunner::runner	INFO	Running command: "/Applications/Firefox.app/Contents/MacOS/firefox-bin" "--marionette" "--remote-debugging-port" "54445" "--remote-allow-hosts" "localhost" "-foreground" "-no-remote" "-profile" "/var/folders/w1/2dngyhy57h996jndj_5w9mn40000gn/T/rust_mozprofileeFU1RU"


2022-10-25 15:57:08.576 plugin-container[11992:23255452] nil host used in call to allowsSpecificHTTPSCertificateForHost
2022-10-25 15:57:08.577 plugin-container[11992:23255452] nil host used in call to allowsAnyHTTPSCertificateForHost:
2022-10-25 15:57:08.581 plugin-container[11992:23255452] nil host used in call to allowsSpecificHTTPSCertificateForHost
2022-10-25 15:57:08.581 plugin-container[11992:23255452] nil host used in call to allowsAnyHTTPSCertificateForHost:
2022-10-25 15:57:08.581 plugin-container[11992:23255458] nil host used in call to allowsSpecificHTTPSCertificateForHost
2022-10-25 15:57:08.581 plugin-container[11992:23255458] nil host used in call to allowsAnyHTTPSCertificateForHost:


1666727828589	Marionette	INFO	Marionette enabled
1666727828624	Marionette	INFO	Listening on port 54452
Read port: 54452


WebDriver BiDi listening on ws://127.0.0.1:54445


1666727828983	RemoteAgent	WARN	TLS certificate errors will be ignored for this session
console.warn: SearchSettings: "get: No settings file exists, new profile?" (new NotFoundError("Could not open the file at /var/folders/w1/2dngyhy57h996jndj_5w9mn40000gn/T/rust_mozprofileeFU1RU/search.json.mozlz4", (void 0)))


DevTools listening on ws://127.0.0.1:54445/devtools/browser/e9975754-aa44-41cd-b6c2-2503ee1ff0e0


# Table

In [214]:
# return the table 
def parse_single_table(df_table:pd.DataFrame) -> pd.DataFrame:
    # fill nan with "Capstone"
    df_table = df_table.fillna("")

    # step 1: find out first non-nan index value
    for i in range(len(df_table.index)):
        if df_table.index[i] is not np.nan:
            first_non_nan_index = i
            break

    # step 2: drop all rows with nan index after first non-nan index
    df_table_dropped_row = df_table[np.append(np.array([True] * first_non_nan_index), df_table.index[first_non_nan_index:].notnull())]

    # step 3: drop all columns with specific values
    columns_name = df_table.columns
    columns_name_drop_0 = [i for i in columns_name if "$" in list(df_table[i])]
    columns_name_drop_1 = [i for i in columns_name if ")" in list(df_table[i])]
    columns_name_drop_2 = [i for i in columns_name if "%" in list(df_table[i])]

    columns_name_drop = set(columns_name_drop_0 + columns_name_drop_1 + columns_name_drop_2)

    df_table_dropped_column = df_table_dropped_row.drop(columns_name_drop, axis=1)

    # step 4: replace "(" in dataframe
    for i in range(len(df_table_dropped_column.columns)):
        try:
            df_table_dropped_column.iloc[:, i] = df_table_dropped_column.iloc[:, i].str.replace("(", "", regex=True)
        except Exception:
            continue

    # step 5: set first_non_nan_index as column name
    def find_row_index_for_column_name(df: pd.DataFrame, first_non_nan_index: int) -> int:
        return next((i for i in range(first_non_nan_index, len(df.index)) if len(set(df.iloc[i])) > 1), first_non_nan_index)

    row_index_for_column_name = find_row_index_for_column_name(df_table_dropped_column, first_non_nan_index)

    df_table_dropped_column.columns = df_table_dropped_column.iloc[row_index_for_column_name]

    # step 6: drop columns with name 'Capstone'
    mapping = [i != '' for i in df_table_dropped_column.columns]
    df_table_dropped_column = df_table_dropped_column.loc[:, mapping]

    # step 7: rename column name
    df_table_dropped_column.columns = range(len(df_table_dropped_column.columns))
    
    #df_table_dropped_column = df_table_dropped_column.T.drop_duplicates().T

    return df_table_dropped_column





def parse_table_from_source(path_source_data: str) -> pd.DataFrame:

    # Prepare table data
    df_table_all = pd.read_html(path_source_data, header=0, index_col=0) 
  
    # step 0: df_table length larger than 5
    df_table_all = [i for i in df_table_all if len(i) > 5]

    df_table_result = [parse_single_table(df_table) for df_table in df_table_all]
    return df_table_result



def get_element_info(element: WebElement)-> list():
    return {"value": element.text,"location": element.location, "size": element.size}

def pause() -> None:
    programPause = input("Press the <ENTER> key to close the selenium webdriver...")
    

def border(elem, driver: webdriver, color:str = "red") -> None:
    driver.execute_script(f"arguments[0].setAttribute(arguments[1], arguments[2])", elem, "style", "padding: 1px; border: 2px solid "+color+"; display: inline-block")


    
def find_page_location(driver: webdriver) -> dict:
    page_breaks = driver.find_elements(By.TAG_NAME, 'hr')
    #print("length of page breaks",len(page_breaks))
    page_number = 1
    # get the range of y for page 1
    page_location = {page_number: [0,page_breaks[0].location["y"]]}
    next_page_start =  page_location[page_number][1] + 2  # plus 2 b/c page break height = 2
    
    # get the range of y for page 2 to  n-1 (n is the last page)
    for hr in page_breaks[1:]:
        page_number += 1
        page_location[page_number]= [next_page_start,hr.location["y"]]
        next_page_start =  page_location[page_number][1] + 2  # plus 2 b/c page break height = 2
    # get the range of y for last page
    page_number += 1
    page_location[page_number]= [next_page_start,float('inf')]
    return page_location

def get_page_number(page_location: dict,element: WebElement) -> int:
    element_y = element.location["y"]
    for i in range(1,len(page_location)+1):
        if( element_y > page_location[i][0] and element_y <page_location[i][1]):
            return i
        
    return None


def get_annotated_label_from_table(driver: webdriver, table: dict, page_num = float("inf"), page_num_y = 0 )-> dict:
    # find all the label element
    found_annotation = table["element"].find_elements(By.TAG_NAME, 'ix:nonnumeric')
    found_annotation += table["element"].find_elements(By.TAG_NAME, 'ix:nonfraction')
    
    # store its location
    annotations = []
    for val in found_annotation: 
        annotations.append({"element": val,"info":get_element_info(val)})
        ##add page number
        annotations[-1]["info"]["page_num"] = page_num
        ##calculate new x and new y base on page
        annotations[-1]["info"]["location"]["y"] -= page_num_y
    return annotations

In [224]:
# create a list of numeric table. Each table has its correspond index to the found table.
numeric_table = [{"index": index, "element": found_table[index]} for index, val in enumerate(table_is_numeric) if val == 0 ]
# dictionary that store the range of y for each page
page_location = find_page_location(driver) 
####color coding#####
for index, val in enumerate(numeric_table) :
    border(val["element"], driver, "Tomato")
    
    val["info"] = get_element_info(val["element"])
    page_num = get_page_number(page_location, val["element"])
    val["info"]["page_num"] = page_num 
    val["info"]["location"]["y"] -=  page_location[page_num][0]

####color coding#####
for table in numeric_table:
    table["annotation"] = get_annotated_label_from_table(driver, table, table["info"]["page_num"],page_location[table["info"]["page_num"]][0] )


In [230]:
numeric_table

[{'index': 9,
  'element': <selenium.webdriver.remote.webelement.WebElement (session="2a93f9f0-37d8-48b4-b149-6eb9495e26b8", element="bafa4e5b-fca7-4bab-8839-63aa69370995")>,
  'info': {'value': 'Three Months Ended Six Months Ended\nJune 30,\n2022 June 30,\n2021 June 30,\n2022 June 30,\n2021\nRevenues\n$ 7,970,141  $ 7,341,777  $ 15,837,908  $ 14,505,059 \nCost of revenues\n4,690,755  4,018,008  8,975,460  7,886,519 \nMarketing\n574,960  603,973  1,130,938  1,116,485 \nTechnology and development\n716,846  537,321  1,374,376  1,062,528 \nGeneral and administrative\n409,297  334,845  807,225  632,041 \nOperating income\n1,578,283  1,847,630  3,549,909  3,807,486 \nOther income (expense):\nInterest expense\n(175,455) (191,322) (363,034) (385,762)\nInterest and other income (expense)\n220,226  (62,519) 415,871  206,567 \nIncome before income taxes\n1,623,054  1,593,789  3,602,746  3,628,291 \nProvision for income taxes (182,103) (240,776) (564,348) (568,563)\nNet income\n$ 1,440,951  $ 1,3

In [229]:

import json

#visual only
print(json.dumps( numeric_table[2], default=lambda o: '<not serializable>',sort_keys=True, indent=4))

{
    "annotation": [
        {
            "element": "<not serializable>",
            "info": {
                "location": {
                    "x": 691,
                    "y": 326
                },
                "page_num": 5,
                "size": {
                    "height": 25.199996948242188,
                    "width": 70.0
                },
                "value": "1,440,951"
            }
        },
        {
            "element": "<not serializable>",
            "info": {
                "location": {
                    "x": 820,
                    "y": 326
                },
                "page_num": 5,
                "size": {
                    "height": 25.199996948242188,
                    "width": 70.0
                },
                "value": "1,353,013"
            }
        },
        {
            "element": "<not serializable>",
            "info": {
                "location": {
                    "x": 949,
                    "y": 32